In [1]:
# importing libraries 
import pandas as pd
import torch
import random
from tqdm import tqdm
from torch.utils.data import DataLoader
import itertools
from transformers import Trainer, TrainingArguments, pipeline, set_seed, AutoTokenizer, BioGptModel, AutoModelForMaskedLM, DataCollatorForLanguageModeling, AutoModel
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import os
from datasets import Dataset

if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

# read the data
df=pd.read_csv("df.csv")
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForMaskedLM.from_pretrained("medicalai/ClinicalBERT")

# preparing the clinical notes for training and fine-tuning
def tokenize(batch):
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=64)

def prepare_data(df):
    texts=list(df["AN_PROC_NAME"])
    # Tokenize texts and create a dataset
    data = {"text": texts}
    dataset = Dataset.from_dict(data)
    # Preprocess the dataset
    dataset = dataset.map(tokenize, batched=True, batch_size=len(texts))
    # Set the format of the dataset
    dataset.set_format("torch", columns=["input_ids", "attention_mask"])
    return(dataset)


/home/charles/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-29 01:23:01.616228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 01:23:02.391412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


2023-09-29 01:23:04.248660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-29 01:23:04.248888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-29 01:23:04.254049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
len(df)

84875

In [3]:
# this finetunes the model with the given parameters
def get_model(name, train_dataset,val_dataset):
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=5, 
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=1000,
        weight_decay=0.01,
        learning_rate=0.0001,
        logging_dir="./logs",
        logging_steps=1000,
        save_strategy="epoch",)
    # Load the ClinicalBERT model with a masked language modeling head
    model = AutoModelForMaskedLM.from_pretrained("medicalai/ClinicalBERT")
    # Define the data collator for masked language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
    # Instantiate the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset, 
        tokenizer=tokenizer,
        data_collator=data_collator,)
    trainer.train()
    trainer.model.save_pretrained(name)
    trainer.tokenizer.save_pretrained(name) 

In [4]:
# this get the embeddings of the fine-tuned model
def get_bioclinicalBERT_updated(texts, name):
    notes = list(texts)
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModel.from_pretrained(name)

    # Check if CUDA is available and if so, move the model to GPU
    if torch.cuda.is_available():
        model = model.to("cuda")

    # Tokenize texts and create input IDs and attention masks
    input_ids, attention_masks = [], []
    for text in notes:
        encoded_input = tokenizer.encode_plus(text, max_length=296, padding='max_length', truncation=True,
                                              return_attention_mask=True, return_tensors='pt')  # change max_length if needed [NOTE: max_length may differ across task]
        input_ids.append(encoded_input['input_ids'])
        attention_masks.append(encoded_input['attention_mask'])

    # Convert lists to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # If CUDA is available, move the tensors to GPU
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")
        attention_masks = attention_masks.to("cuda")

    # Batch size for processing
    batch_size = 32
    # Initialize an empty list to store embeddings
    embeddings = []
    # Process input IDs and attention masks in batches
    for i in tqdm(range(0, len(input_ids), batch_size)):
        input_ids_batch = input_ids[i:i+batch_size]
        attention_masks_batch = attention_masks[i:i+batch_size]

        with torch.no_grad():
            # Get the model's output
            output = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)

        # Retrieve the last_hidden_state from the output
        last_hidden_state = output.last_hidden_state

        # Optionally, you can obtain the pooled_output by averaging or max-pooling the last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)

        # If CUDA is available, move the embeddings back to CPU
        if torch.cuda.is_available():
            pooled_output = pooled_output.to("cpu")

        # Add the embeddings to the list
        embeddings.extend(pooled_output.numpy())

    embeddings = np.array(embeddings)
    return embeddings


In [5]:
def K_fold_val(outcome_col, df):
    #print(outcome_col)
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df,df["PE"])):
        # Split the data into train/test sets
        train, test = df.iloc[train_index], df.iloc[test_index]
        # Create a new column for embeddings based on the training set only
        if outcome_col=="postop_del":
            name=f"{outcome_col}_fold_{i}"
        else:
            name=f"fold_{i}"
        #name=f"{outcome_col}_fold_{i}"
        i=i+1
        train_data=((train.sample(frac=7/8,random_state=42)))
        val_data = train[~train.index.isin(train_data.index)]
        train_data=train_data.reset_index(drop=True)
        val_data=val_data.reset_index(drop=True)
        train_dataset = prepare_data(train_data)
        val_dataset = prepare_data(val_data)
        if os.path.isdir(name):
            pass
        else:
            get_model(name, train_dataset,val_dataset)
        # print(f"{name}.pickle")
        if os.path.isfile(f"{name}.pickle"): 
            # print("file present")
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_bioclinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_bioclinicalBERT_updated(test["AN_PROC_NAME"], name)
        
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)


        y_train = train[outcome_col]
        y_test = test[outcome_col]
        # Prepare training and testing data
        model = xgb.XGBClassifier(random_state=42,tree_method = "hist", device = "cuda")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [6]:
results=K_fold_val("death_in_30", df)

Map: 100%|██████████| 8488/8488 [00:00<00:00, 9098.55 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits


/home/charles/.local/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:23:24] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   8.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   8.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   9.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   9.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   8.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   8.1s
[CV] END .learning_rate=0.1,

Map: 100%|██████████| 8488/8488 [00:01<00:00, 8190.66 examples/s]


...

Map: 100%|██████████| 8488/8488 [00:00<00:00, 8707.43 examples/s]
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1000,1.244300
2000,0.831200
3000,0.707600
4000,0.613200
5000,0.538800
6000,0.526800
7000,0.477600
8000,0.441300
9000,0.429600
10000,0.405700


/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input 

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [4:12:54, 3034.81s/it]


In [7]:
print(results) 

metrics: Mean AUROC: 0.8597155762865905, SD AUROC: 0.008951979663525456, Mean AUPRC: 0.15812862774179007, SD AUPRC: 0.012175168850973847


## Part II: DVT

In [8]:
results=K_fold_val("DVT", df)

Map: 100%|██████████| 8488/8488 [00:01<00:00, 7983.97 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 9341.86 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15955.85 examples/s]


...

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15652.07 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 9247.16 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [42:36, 511.32s/it]


In [9]:
print(results) 

metrics: Mean AUROC: 0.7667238381792785, SD AUROC: 0.03434754832610927, Mean AUPRC: 0.023397005534061228, SD AUPRC: 0.006455298076554233


## Part III: PE

In [10]:
results=K_fold_val("PE", df)

Map: 100%|██████████| 8488/8488 [00:00<00:00, 16141.31 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15683.10 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 16026.26 examples/s]


...

Map: 100%|██████████| 8488/8488 [00:01<00:00, 8182.63 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15703.12 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [42:16, 507.20s/it]


In [11]:
print(results) 

metrics: Mean AUROC: 0.7296351317438378, SD AUROC: 0.031133623384732897, Mean AUPRC: 0.014410968389401726, SD AUPRC: 0.0026065154983013323


## Part 4: PNA outcome

In [12]:
results=K_fold_val("PNA", df)

Map: 100%|██████████| 8488/8488 [00:00<00:00, 16464.62 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 16204.44 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 16047.70 examples/s]


...

Map: 100%|██████████| 8488/8488 [00:00<00:00, 14764.40 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 9611.04 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [44:58, 539.64s/it]


In [13]:
print(results) 

metrics: Mean AUROC: 0.8086300197909537, SD AUROC: 0.022559581535651674, Mean AUPRC: 0.04727313290874025, SD AUPRC: 0.012111977972257191


## Part 5: post-AKI status

In [14]:
results=K_fold_val("post_aki_status", df)

Map: 100%|██████████| 8488/8488 [00:00<00:00, 16254.71 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15319.79 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15366.27 examples/s]


...

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15919.53 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Map: 100%|██████████| 8488/8488 [00:00<00:00, 15592.34 examples/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [42:02, 504.49s/it]


In [15]:
print(results)

metrics: Mean AUROC: 0.8353435799166251, SD AUROC: 0.0015721377531448181, Mean AUPRC: 0.4788010231026291, SD AUPRC: 0.00722540268710723


## Part 6: Delirium

In [16]:
df = df.dropna(subset=['postop_del']).reset_index(drop=True)
df["postop_del"]=df["postop_del"].astype(int)
results=K_fold_val("postop_del", df)

Map: 100%|██████████| 1218/1218 [00:00<00:00, 3887.42 examples/s]


/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1000,1.027100
2000,0.644000


/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input 

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Map: 100%|██████████| 1218/1218 [00:00<00:00, 3025.16 examples/s]
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1000,0.989300
2000,0.664500


/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/charles/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input 

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [1:48:15, 1299.13s/it]


In [17]:
print(results) 

metrics: Mean AUROC: 0.6978988301369456, SD AUROC: 0.008634071803510064, Mean AUPRC: 0.670679627127599, SD AUPRC: 0.014350607405809531
